# LSTM using raw data

# Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import Sigmoid, Softmax
import sys
sys.path.append('../')
from labelled_data.tools.load_data import data_loader
from labelled_data.tools.load_data import data_generator
from torch.nn.functional import normalize, binary_cross_entropy, binary_cross_entropy_with_logits
from torch.autograd import Variable

figure_path = '../figures/'

In [ ]:
use_cuda=True
def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

cuda=False
if torch.cuda.is_available():  
  dev = "cuda:0" 
  cuda=True
else:
  dev = "cpu"
device = torch.device(dev)

# Raw data or chunked data

In [ ]:
chunks = False

# Define data loader

In [ ]:
X_train, X_test, X_valid, y_train, y_test, y_valid = data_loader(chunks=chunks, no_files=10)
train_gen = data_generator(X_train, y_train, sequence_length=500)

In [ ]:
data, labels = train_gen.__next__()

In [ ]:
data.shape

# Define model

In [ ]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, lstm_hidden_size, linear_hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.lstm_hidden_size = lstm_hidden_size #hidden state

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=lstm_hidden_size,
                          num_layers=num_layers, batch_first=True, dropout=0.4) #lstm
        self.linear_1 = nn.Linear(lstm_hidden_size, linear_hidden_size) #fully connected last layer
        self.dropout = nn.Dropout(0.4)
        #self.linear_2 = nn.Linear(linear_hidden_size, int(linear_hidden_size/2)) #fully connected last layer
        self.linear_out = nn.Linear(int(linear_hidden_size), num_classes) #fully connected last layer
        self.sigmoid = Sigmoid()
        self.relu = nn.ReLU()

    def forward(self,x):
        # Propagate input through LSTM
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.lstm_hidden_size, device=x.device)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.lstm_hidden_size, device=x.device)) #internal state

        x, (h, c) = self.lstm(x, (h_0, c_0)) # lstm with input, hidden, and internal state
        
        x = x.reshape(x.shape[0]*x.shape[1], x.shape[2]) # reshaping the data for Dense layer next

        x = self.linear_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        #x = self.linear_2(x)
        #x = self.relu(x)
        x = self.linear_out(x) 

        out = self.sigmoid(x)
        return out

## Hyper parameters

In [ ]:
sequence_length = 1
num_epochs = 200 # 1000 epochs
learning_rate = 0.001

input_size = 1 # number of features
lstm_hidden_size = 40 # number of features in hidden state
linear_hidden_size = 40 # number of features in hidden state
num_layers = 3 # number of stacked lstm layers

num_classes = 1 # number of output classes

no_batches = 50000
no_files = 20 # no. data files

 ## Initialize

In [ ]:
lstm = LSTM(num_classes, input_size, lstm_hidden_size, linear_hidden_size, num_layers) #our lstm class
lstm = get_variable(lstm)

## Loss function and optimizer

In [ ]:
criterion = torch.nn.BCELoss() # cross validation
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#criterion = torch.nn.MSELoss()    # mean-squared error for regression
#optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# Train the model

In [ ]:
def evaluate_result(output):
    output[output > 0.5] = 1.
    output[output < 0.5] = 0.
    return output

In [ ]:
epoch_training_loss = []
epoch_validation_loss = []
epoch_training_acc = []
epoch_validation_acc = []

X_train, X_test, X_valid, y_train, y_test, y_valid = data_loader(chunks=chunks, no_files=no_files)
print('training files', len(X_train))
print('validation files', len(X_valid))
print('test files', len(X_test))

for epoch in range(num_epochs):
  training_loss = 0
  validation_loss = 0
  training_correct = 0
  training_all = 0
  validation_correct = 0
  validation_all = 0
  
  train_gen = data_generator(X_train, y_train, sequence_length=sequence_length)
  test_gen = data_generator(X_test, y_test, sequence_length=sequence_length)
  valid_gen = data_generator(X_valid, y_valid, sequence_length=sequence_length)

  dict = {
      'no_train' : len(X_train),
      'train': train_gen,
      'no_test' : len(X_test),
      'test': test_gen,
      'no_valid' : len(X_valid),
      'valid': valid_gen,
  }

  for _ in range(len(X_valid)):
      i_batch = 0

      data, labels = valid_gen.__next__()
      data = get_variable(data*1000)
      labels = get_variable(labels)

      while data.shape[0] > i_batch + no_batches:
        batch_data = data[i_batch:(i_batch+no_batches)]
        batch_labels = labels[(i_batch*sequence_length):(i_batch+no_batches)*sequence_length]

        lstm.eval()
        outputs = lstm.forward(batch_data) #forward pass
        loss = criterion(outputs, batch_labels)
        validation_loss += loss.item()

        validation_correct += (evaluate_result(outputs) == batch_labels).float().sum()
        validation_all += len(batch_labels)
        i_batch += no_batches

  for _ in range(len(X_train)):
      i_batch = 0
      
      data, labels = train_gen.__next__()
      data = get_variable(data*1000)
      labels = get_variable(labels)
      if epoch == 0:
        print('Number of batch loops', data.shape[0]/no_batches)

      while data.shape[0] > i_batch + no_batches:
        batch_data = data[i_batch:(i_batch+no_batches)]
        batch_labels = labels[(i_batch*sequence_length):(i_batch+no_batches)*sequence_length]

        lstm.train()

        train_outputs = lstm.forward(batch_data) #forward pass
        optimizer.zero_grad() #caluclate the gradient, manually setting to 0

        # obtain the loss function
        loss = criterion(train_outputs, batch_labels)
        loss.backward(retain_graph=True) #calculates the loss of the loss function
        training_loss += loss.item()

        training_correct += (evaluate_result(train_outputs) == batch_labels).float().sum()
        training_all += len(batch_labels)

        optimizer.step() #improve from loss, i.e backprop

        i_batch += no_batches

  validation_acc = validation_correct / float(validation_all)
  training_acc = training_correct / float(training_all)
  #if epoch % 10 == 0:
  print("Epoch: %d, training loss: %1.5f, validation loss: %1.5f, training acc: %1.5f, , validation acc: %1.5f" % (epoch, training_loss/len(X_train), validation_loss/len(X_valid), training_acc, validation_acc))

  epoch_validation_loss.append(validation_loss/len(X_valid))
  epoch_validation_acc.append(validation_acc)
  epoch_training_loss.append(training_loss/len(X_train))
  epoch_training_acc.append(training_acc)

In [ ]:
torch.save(lstm.state_dict(), '../models/model')

In [ ]:
train_gen = data_generator(X_train, y_train)

In [ ]:
plt.figure(figsize=(40,5))
data, labels = train_gen.__next__()
data = data.detach().numpy()[:,0,0]
#plt.plot(data[int(len(data)*0.25):int(len(data)*0.75)])
plt.plot(data[5000:50000]*1000)

if labels.sum() > 0:
#plt.plot(labels[int(len(data)*0.25):int(len(data)*0.75)]*1330)
    plt.plot(labels[5000:50000]*0.2)
#plt.gca().set_ylim([1300,1500])
#plt.gca().set_xlim([1610000,len(data)-860000])

plt.show()

In [ ]:
plt.plot(np.linspace(0, len(epoch_validation_loss)-1, len(epoch_validation_loss)), epoch_validation_loss, label='validation loss')
plt.plot(np.linspace(0, len(epoch_training_loss)-1, len(epoch_training_loss)), epoch_training_loss, label='training loss')
plt.gca().set_ylabel('Loss')
plt.gca().set_xlabel('Epochs')
plt.legend()
plt.savefig(figure_path + 'validation.png', bbox_inches='tight')
plt.show()

In [ ]:
epoch_validation_acc = [acc.cpu() for acc in epoch_validation_acc]
epoch_training_acc = [acc.cpu() for acc in epoch_training_acc]
test_gen = data_generator(X_test, y_test)

plt.plot(np.linspace(0, len(epoch_validation_acc)-1, len(epoch_validation_acc)), epoch_validation_acc, label='validation acc')
plt.plot(np.linspace(0, len(epoch_training_acc)-1, len(epoch_training_acc)), epoch_training_acc, label='training acc')
plt.gca().set_ylabel('Accuracy')
plt.gca().set_xlabel('Epochs')
plt.legend()
plt.savefig(figure_path + 'accuracy.png', bbox_inches='tight')
plt.show()

# Run the model

In [ ]:
data, labels = test_gen.__next__()
#train_predict = lstm(data.cuda())#forward pass
train_predict = lstm(data.cuda())#forward pass
data_predict = train_predict.data.cpu().numpy() #numpy conversion
data_predict[data_predict > 0.5] = 1
data_predict[data_predict < 0.5] = 0

dataY_plot = labels

plt.figure(figsize=(10,6)) #plotting
#plt.axvline(x=40000, c='r', linestyle='--') #size of the training set

plt.plot(dataY_plot, label='Actual Data') #actual plot
plt.plot(data_predict, label='Predicted Data') #predicted plot
plt.title('Time-Series Prediction')
plt.legend()
plt.show()